In [1]:
from pathlib import Path
import pandas as pd
import plotly.graph_objects as go
import matplotlib
from matplotlib import cm

from hydra.utils import split_dupes, write
from hydra.plotly_charts import PlotlyPriceChart, supersample_data


# Slider for parent window
# Slider for child window
# Slider for visible range


# SYMBOLS = ["circle", "square", "diamond-tall", "diamond-wide"]
# COLORS = ["red", "blue", "lime", "goldenrod"]
pair = "BTCUSD"

HEATMAP_COLORS = "afmhot_r"
chart_settings = dict(
    trendline=False, ifft_price=False, extrapolated=True, showHeatMap=False
)


window_and_timespan = "90-365"
outputDir = Path("../output") / f"enviro-chunky-{window_and_timespan}"


In [2]:
rootWindowList = sorted({int(file.stem) for file in outputDir.rglob(f"year=*/**/*[!.xtrp].parq")})
print('rootWIndowList', rootWindowList)

ROOT_WINDOW = "1"


def loadOutputs(rootWindow="*"):
    fname = f"{rootWindow}[!.xtrp]" if rootWindow == '*' else rootWindow
    outputFiles = list(outputDir.rglob(f"year=*/**/{fname}.parq"))
    return pd.concat([pd.read_parquet(file) for file in outputFiles])

def getOutputs(rootWindow="*"):
    outputs = loadOutputs(rootWindow)    
    outputs = outputs.sort_index()[
        [
            "minPerCycle",
            "deviance",
            "ifft_extrapolated_wavelength",
            "ifft_extrapolated_amplitude",
            "ifft_extrapolated_deviance",
            "first_extrapolated",
            "first_extrapolated_date",
            "first_extrapolated_isup",
            "startDate",
            "endDate",
            "window",
            "window_original",
            "trend_deviance",
            "trend_slope",
            "trend_intercept",
            "rootNumber",
        ]
    ]

    outputs["text"] = (
        outputs["endDate"].astype(str)
        + "("
        + (outputs["first_extrapolated_date"] - outputs["endDate"]).astype(str)
        + ")<br>isup = "
        + outputs["first_extrapolated_isup"].astype(str)
        + "<br>🌊"
        + outputs["ifft_extrapolated_wavelength"].astype(str)
        + "<br>🔊"
        + outputs["ifft_extrapolated_amplitude"].astype(str)
    )
    return outputs.set_index("first_extrapolated_date", drop=False)

outputs = getOutputs(ROOT_WINDOW)
print(outputs.columns)

startDate = outputs["startDate"].min()
endDate = outputs["endDate"].max()



rootWIndowList [1]
Index(['minPerCycle', 'deviance', 'ifft_extrapolated_wavelength',
       'ifft_extrapolated_amplitude', 'ifft_extrapolated_deviance',
       'first_extrapolated', 'first_extrapolated_date',
       'first_extrapolated_isup', 'startDate', 'endDate', 'window',
       'window_original', 'trend_deviance', 'trend_slope', 'trend_intercept',
       'rootNumber', 'text'],
      dtype='object')


In [3]:
print(getOutputs(1)[["first_extrapolated", "text", "endDate"]])
# print(getOutputs(2)[["first_extrapolated", "text", "endDate"]])

                            first_extrapolated  \
first_extrapolated_date                          
2018-01-01 00:00:00.000000        16245.763578   
2018-01-03 02:35:00.000000        16787.234250   
2018-01-03 16:33:49.090909        16979.636113   
2018-01-04 09:41:00.000000        17142.169613   
2018-02-09 17:20:00.000000        20752.032988   
...                                        ...   
2018-12-31 16:02:00.000000         3053.017219   
2019-01-02 00:59:31.764705         2979.125842   
2018-12-31 04:28:00.000000         3082.042220   
2019-01-02 02:05:00.000000         2988.347700   
2019-01-02 07:58:13.846153         3000.343520   

                                                                         text  \
first_extrapolated_date                                                         
2018-01-01 00:00:00.000000  2018-01-01(0 days 00:00:00)<br>isup = True<br>...   
2018-01-03 02:35:00.000000  2018-01-01(2 days 02:35:00)<br>isup = True<br>...   
2018-01-03 16:33:49.09090

In [4]:
# startDate = "2017-05-15 00:00"
# endDate = "2020-05-16 00:00"
print(startDate, endDate)


# agg_chart_data = pd.DataFrame(
#     {
#         "extrapolated": [chart["first_extrapolated"][0] for chart in charts],
#         "extrapolated_date": [chart["first_extrapolated"][1] for chart in charts],
#         "is_up": [chart["first_extrapolated"][2] for chart in charts],
#         # "distance": [
#         #     (chart["first_extrapolated"][1] - chart["endDate"]).floor(
#         #         extrapolated_distance_bucket_size
#         #     )
#         #     for chart in charts
#         # ],
#         "startDate": [chart["startDate"] for chart in charts],
#         "endDate": [chart["endDate"] for chart in charts],
#         # "distance_":[(chart['first_extrapolated'][1] - chart['endDate']) for chart in charts],
#         "text": [
#             f"{chart['endDate']} ({(chart['first_extrapolated'][1] - chart['endDate'])})<br>isup = {chart['first_extrapolated'][2]}<br>🌊{chart['ifft_extrapolated_wavelength']}<br>🔊{chart['ifft_extrapolated_amplitude']}"
#             for chart in charts
#         ],
#     }
# )
# print(agg_chart_data)
# agg_chart_data.set_index("extrapolated_date", inplace=True)
# agg_data_by_distance = {
#     name: {"up": group.loc[group["first_extrapolated_isup"]], "down": group.loc[~group["first_extrapolated_isup"]]}
#     for name, group in agg_chart_data.groupby("distance")
# }


# extra_up = agg_chart_data.loc[agg_chart_data["is_up"]]
# extra_down = agg_chart_data.loc[~agg_chart_data["is_up"]]


# print(trendlines)
priceLoc = (2, 1) if chart_settings["showHeatMap"] else (1, 1)
priceChart = PlotlyPriceChart(pair, startDate, endDate, loc=priceLoc)


def add_trace(
    data,
    chartParams={},
    interval=1,
    useIntervals=[],
    alwaysUseData=False,
    approximateIntervals=False,
    onlySlice=False,
    fields={"x": "index", "y": "values"},
    metaHandler=None,
):
    if useIntervals == False:
        priceChart.add_trace(
            go.Scatter(x=data.index, y=data.values, **chartParams),
            onlySlice=onlySlice,
            fields=fields,
            metaHandler=metaHandler,
            loc=priceLoc,
        )
        return

    supersampled = supersample_data(
        data,
        interval,
        useIntervals,
        alwaysUseData=alwaysUseData,
        approximate=approximateIntervals,
    )
    priceChart.add_trace(
        go.Scatter(
            x=supersampled[1440].index, y=supersampled[1440].values, **chartParams
        ),
        data=supersampled,
        onlySlice=onlySlice,
        fields=fields,
        metaHandler=metaHandler,
        loc=priceLoc,
    )


# if chart_settings["ifft_price"]:
#     for index in range(traceCount):
#         if index in chart_data:
#             data = pd.concat(chart_data[index]["ifft_price"])
#             add_trace(data, chartParams={"mode": "lines", "name": f"ifft-{index}"})
# if chart_settings["trendline"]:
#     for index in range(traceCount):
#         if index in chart_data:
#             data = pd.concat(chart_data[index]["trendline"])
#             add_trace(
#                 data,
#                 chartParams={
#                     "mode": "markers",
#                     "name": f"trend-{index}",
#                     # "marker_symbol":SYMBOLS[index]
#                 },
#             )

# color = cm.get_cmap(HEATMAP_COLORS, len(agg_data_by_distance))
# useIntervals = [
#     srate
#     for srate in [1, 5, 15, 60, 720, 1440]
#     if pd.Timedelta(f"{srate * 250 / 30}min") < window
# ]
useIntervals = [1, 5, 15, 60, 720, 1440]
print(useIntervals)


if chart_settings["extrapolated"]:

    def extrapolated_metaHandlerFactory(directionUp=True):
        lastRootWindow = ROOT_WINDOW

        def extrapolated_metaHandler(lastData, meta):
            nonlocal lastRootWindow

            nextRootWindow = meta.get("rootWindow", None)
            write(
                f"extrapolated_metaHandler {nextRootWindow=} {lastRootWindow=} {nextRootWindow is not None and nextRootWindow != lastRootWindow}"
            )

            if nextRootWindow is not None and nextRootWindow != lastRootWindow:
                lastRootWindow = nextRootWindow
                # Load Data
                nextData = getOutputs(nextRootWindow)
                write("extrapolated_metaHandler", "Data Loaded")
                # Process Data
                nextData = (
                    nextData.loc[nextData["first_extrapolated_isup"]]
                    if directionUp
                    else nextData.loc[~nextData["first_extrapolated_isup"]]
                )
                
                nextData = nextData[["first_extrapolated", "text", "endDate"]]
                nextData = supersample_data(
                    nextData,
                    1,
                    useIntervals,
                    alwaysUseData=True,
                )
                write("extrapolated_metaHandler", "Data Processed", lastData[1].equals(nextData[1]))
                
                return nextData
            else:
                return lastData

        return extrapolated_metaHandler

    up = outputs.loc[outputs["first_extrapolated_isup"]]
    down = outputs.loc[~outputs["first_extrapolated_isup"]]
    # hex = matplotlib.colors.rgb2hex(color(num))

    add_trace(
        up[["first_extrapolated", "text", "endDate"]],
        chartParams={
            "mode": "markers",
            "name": f"xtra-u ",
            "marker_size": 10,
            "marker_symbol": "triangle-up",
            "marker_color": "green",  # hex,
            "text": "text",
        },
        useIntervals=useIntervals,
        alwaysUseData=True,
        # approximateIntervals=True,
        onlySlice=True,
        fields={"x": "index", "y": "first_extrapolated", "text": "text"},
        metaHandler=extrapolated_metaHandlerFactory(directionUp=True),
    )
    add_trace(
        down[["first_extrapolated", "text", "endDate"]],
        chartParams={
            "mode": "markers",
            "name": f"xtra-d",
            "marker_size": 10,
            "marker_symbol": "triangle-down",
            "marker_color": "red",  # hex,
            "text": "text",
        },
        useIntervals=useIntervals,
        alwaysUseData=True,
        # approximateIntervals=True,
        onlySlice=True,
        fields={"x": "index", "y": "first_extrapolated", "text": "text"},
        metaHandler=extrapolated_metaHandlerFactory(directionUp=False),
    )

if chart_settings["showHeatMap"]:
    y = complexity.index.tolist()
    # y = [str(float(val).round(4)) for val in y]
    priceChart.add_trace(
        go.Heatmap(
            z=complexity.values.tolist(),
            x=complexity.columns.tolist(),
            y=y,
            showscale=False,
        ),
        loc=(1, 1),
        traceArgs={},
    )
    # priceChart.figure.add_vline(x=startDate, line_width=3)

# Create and add slider
time_slider_steps = []
for i in range(100):
    step = dict(
        method="relayout",
        args=[
            # {"title": f"{i:02d}" + f"% from start (0 = show all)"},
            # {"title": f"{i:02d}% from start (0 = show all)", "meta": {"timeSlider":i}}
            # {"title": f"{i:02d}% from start (0 = show all)", "meta.timeSlider":i}
            {"meta": {"timeSlider": i}}
            # {"meta.timeSlider": i}
        ],  # layout attribute
        label=i,
    )
    time_slider_steps.append(step)

# frequency_slider_steps = []
# for i in range(len(priceChart.figure.data)):
#     step = dict(
#         method="update",
#         args=[{"visible": [False] * len(priceChart.figure.data)},
#               {"title": "Slider switched to step: " + str(i)}],  # layout attribute
#     )
#     step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
#     time_slider_steps.append(step)

rootWindowSliderSteps = [
    dict(
        method="relayout",
        args=[
            # {"meta.rootWindow": rootWindow}
            {"meta": {"rootWindow": rootWindow}}
        ],  # layout attribute
        label=rootWindow,
    )
    for rootWindow in rootWindowList
]


sliderPad = 150 if chart_settings["showHeatMap"] else 0
sliders = [
    dict(
        active=50,
        currentvalue={"suffix": "%Time: "},
        pad={"t": sliderPad},
        steps=time_slider_steps,
    ),
    dict(
        active=0,
        currentvalue={"prefix": "Root Window:"},
        pad={"t": 150},
        steps=rootWindowSliderSteps,
    ),
]
# ,dict(
#     active=10,
#     currentvalue={"prefix": "Frequency: "},
#     pad={"t": 250},
#     steps=frequency_slider_steps
# )


windowSize = pd.to_timedelta("1d")
priceChart.render(sliders=sliders, zoomStart=startDate, zoomEnd=startDate + windowSize)


2017-10-03 00:01:00 2018-12-31 00:00:00
registering handler 0 False
[1, 5, 15, 60, 720, 1440]
registering handler 1 True
registering handler 2 True


FigureWidget({
    'data': [{'close': array([4309. , 4392.6, 4376.7, 4385. , 4390. , 4387.3, 4404. , 4411.8, 4…